# Fully entangled PQC using Amazon Braket

Very similar example [here](https://github.com/amazon-braket/amazon-braket-examples/tree/main/examples/hybrid_quantum_algorithms/QAOA)
and code adapts from it.

In [ ]:
# general imports
import matplotlib.pyplot as plt

# magic word for producing visualizations in notebook
%matplotlib widget

from braket.circuits import Circuit
from braket.circuits import FreeParameter
from braket.devices import LocalSimulator
from braket.circuits.observables import Z

import numpy as np
from scipy.optimize import minimize

In [ ]:
# Ensure consistent results
np.random.seed(0)

# Set up device: Local Simulator
device = LocalSimulator()

In [ ]:
def rotations(wire, params):
    circuit = Circuit()
    circuit.rz(wire, params[0])
    circuit.ry(wire, params[1])
    circuit.rz(wire, params[2])
    return circuit

def entangle(n_qubits):
    circuit = Circuit()
    if n_qubits <= 1:
        return circuit
    for ii in range(n_qubits):
        circuit.cnot(ii, (ii + 1) % n_qubits)
    return circuit

def training_layer(n_qubits, params):
    circuit = Circuit()
    for ii in range(n_qubits):
        circuit.add(rotations(ii, params[ii,:]))
    circuit.add(entangle(n_qubits))
    return circuit

def encoding_layer(n_qubits, params):
    circuit = Circuit()
    for ii in range(n_qubits):
        circuit.rx(ii, params[ii])
    return circuit

In [ ]:
def enc_params(n_qubits, layers):
    params = np.empty((n_qubits), object)
    for nn in range(n_qubits):
        name = f"enc_q_{nn}"
        params[nn] = FreeParameter(name)
    return params

def rot_params(n_qubits, layers):
    params = np.empty((layers, n_qubits, 3), object)
    for ll in range(layers):
        for nn in range(n_qubits):
            for aa in range(3):
                name = f"rot_l_{ll}_q_{nn}_a_{aa}"
                params[ll, nn, aa] = FreeParameter(name)
    return params

In [ ]:
def circuit(n_qubits, layers):
    circuit = Circuit()
    free_enc_params = enc_params(n_qubits, layers)
    free_rot_params = rot_params(n_qubits, layers)
    for ii in range(layers):
        # data reencoding every layer
        circuit.add(encoding_layer(n_qubits, free_enc_params))
        circuit.add(training_layer(n_qubits, free_rot_params[ii]))
    return circuit

In [ ]:
n_qubits = 3
layers = 3

In [ ]:
print(circuit(n_qubits, layers))

In [ ]:
def init_params(n_qubits, layers, angles):
    params = {}
    for nn in range(n_qubits):
        name = f"enc_q_{nn}"
        params[name] = angles[nn]
    for ll in range(layers):
        for nn in range(n_qubits):
            for aa in range(3):
                name = f"rot_l_{ll}_q_{nn}_a_{aa}"
                params[name] = np.random.uniform(0, 2*np.pi)
    return params

def params_dict2list(n_qubits, layers, params_dict):
    params_list = []
    for nn in range(n_qubits):
        name = f"enc_q_{nn}"
        params_list.append(params_dict[name])
    for ll in range(layers):
        for nn in range(n_qubits):
            for aa in range(3):
                name = f"rot_l_{ll}_q_{nn}_a_{aa}"
                params_list.append(params_dict[name])
    return params_list

def params_list2dict(n_qubits, layers, params_list):
    params_dict = {}
    ii = 0
    for nn in range(n_qubits):
        name = f"enc_q_{nn}"
        params_dict[name] = params_list[ii]
        ii += 1
    for ll in range(layers):
        for nn in range(n_qubits):
            for aa in range(3):
                name = f"rot_l_{ll}_q_{nn}_a_{aa}"
                params_dict[name] = params_list[ii]
                ii += 1
    return params_dict

def params_bounds(n_qubits, layers):
    return [(0, 2 * np.pi) for _ in range(layers * n_qubits * 3)]

In [ ]:
angles = np.tile(1.0, (3))
params0 = init_params(n_qubits, layers, angles)
params_list = params_dict2list(n_qubits, layers, params0)
params_list2dict(n_qubits, layers, params_list) == params0

In [ ]:
device.run(circuit(n_qubits, layers), shots = 1000, inputs = init_params(n_qubits, layers, angles)).result()

In [ ]:
def expectation(probs):
    # use expectation value to predict efficiency
    val = 0
    for key, value in probs.items():
        eig = 1
        for char in key:
            if char == "1":
                eig *= -1
        val += eig * value
    return val

def linmap(value, amin, amax, bmin, bmax):
    return bmin + (bmax - bmin)/(amax - amin) * (value - amin)

def map_expval(expval, start, stop):
    return linmap(expval, -1, 1, start, stop)

In [ ]:
def simulate_efficency(params, n_qubits, layers, circuit, device, shots):
    params_dict = params_list2dict(n_qubits, layers, params)
    # classically simulate the circuit
    # set the parameter values using the inputs argument
    # execute the correct device.run call depending on whether the backend is local or cloud based
    if isinstance(device, LocalSimulator):
        task = device.run(circuit, inputs=params_dict, shots=shots)
    else:
        task = device.run(
            circuit,
            shots=shots,
            inputs=params_dict,
            poll_timeout_seconds=3 * 24 * 60 * 60,
        )

    # get result for this task
    result = task.result()
    if shots == 0:
        # results assumed in index 0
        expval = result.values[0]
    else:
        measurement_probabilities = result.measurement_probabilities
        expval = expectation(measurement_probabilities)

    eff = map_expval(expval, 0, 1)

    return eff

In [ ]:
def objective_function(params, target, n_qubits, layers, circuit, device, shots, tracker, verbose):
    tracker.update({"count": tracker["count"] + 1})
    if verbose:
        print("=" * 80)
        print("Iteration step. Cycle:", tracker["count"])

    # minimize RMS difference with target dataset
    diff = []
    for ii in range(target.shape[0]):
        angle = target[ii,0]
        eff = target[ii,1]
        angles = np.tile(angle, (3))
        params_list = np.concatenate((angles, params))
        simeff = simulate_efficency(params_list, n_qubits, layers, circ, device, shots)
        diff.append(simeff - eff)
    s = np.asarray(diff)
    s = np.square(s)
    rms = np.sqrt(np.mean(s))

    if verbose:
        print("RMS:", rms)

    # update tracker
    tracker["rms"].append(rms)
    tracker["params"].append(params)
    
    return rms

In [ ]:
circ = circuit(n_qubits, layers)
circ.expectation(Z(0) @ Z(1) @ Z(2))
angles = np.tile(1.0, (3))
params = init_params(n_qubits, layers, angles)
params = np.asarray(params_dict2list(n_qubits, layers, params))
simulate_efficency(params, n_qubits, layers, circ, device, 100)

## Check convergence of shot count

In [ ]:
def shot_conv(shots_list, angle, n_qubits, layers, circ, device):
    angles = np.tile(angle, (3))
    params = init_params(n_qubits, layers, angles)
    params = np.asarray(params_dict2list(n_qubits, layers, params))
    eff = []
    for shots in shots_list:
        eff.append(simulate_efficency(params, n_qubits, layers, circ, device, shots))
    return eff

In [ ]:
X = range(0,10000,200)
Y = shot_conv(X, 1.0, n_qubits, layers, circ, device)

In [ ]:
plt.clf()
plt.plot(X, Y)
plt.show()

# Train PQC

In [ ]:
def train(func, n_qubits, layers, device, shots, tracker, target, options, opt_method="cobyla", verbose=True):
    """Function to train VQE"""
    print("Starting the training.")

    print("=" * 80)
    print(f"OPTIMIZATION for {n_qubits} qubits, {layers} layers")

    if not verbose:
        print('Param "verbose" set to False. Will not print intermediate steps.')
        print("=" * 80)

    # randomly initialize variational parameters
    params = [np.random.uniform(0, 2*np.pi) for _ in range(n_qubits * layers * 3)]

    # set bounds for search space
    bounds = params_bounds(n_qubits, layers)

    circ = circuit(n_qubits, layers)

    # run classical optimization (example: method='Nelder-Mead')
    result = minimize(
        func,
        np.asarray(params),
        args=(target, n_qubits, layers, circ, device, shots, tracker, verbose),
        bounds=bounds,
        options=options,
        method=opt_method,
    )

    # store result of classical optimization
    cost = result.fun
    print("Final cost:", cost)
    result_angles = result.x
    print("Final angles:", result_angles)
    print("Training complete.")

    return cost, result_angles, tracker

In [ ]:
# set tracker to keep track of results
tracker = {
    "count": 0,  # Elapsed optimization steps
    "rms": [],  # RMS at each step
    "params": [],  # Track parameters
}

options = {"maxiter": 20}
target = np.loadtxt("multall_runs.csv", delimiter=",", usecols=(3,12), skiprows = 1)
for ii in range(target.shape[0]):
    target[ii,0] = linmap(target[ii,0], -20, 20, 0, 2*np.pi)

In [ ]:
train(objective_function, n_qubits, layers, device, 2000, tracker, target, options)